```{contents}
```
## **Interactive Debugging in LangGraph**

**Interactive debugging** in LangGraph refers to the ability to **inspect, pause, modify, and resume** graph execution at runtime while observing the evolution of **state, decisions, and control flow**.
It is essential for building **reliable, interpretable, and production-grade LLM systems**.

---

### **1. Why Interactive Debugging Is Necessary**

LangGraph systems are:

* **Stateful**
* **Non-linear**
* **LLM-driven (non-deterministic)**
* **Multi-agent and cyclic**

Traditional debugging tools (breakpoints, logs) are insufficient.

LangGraph introduces **execution-aware debugging**:

| Capability         | Purpose                          |
| ------------------ | -------------------------------- |
| Inspect live state | Understand reasoning & decisions |
| Pause execution    | Safely intervene                 |
| Replay execution   | Reproduce failures               |
| Modify state       | Correct behavior                 |
| Trace control flow | Observe routing decisions        |

---

### **2. Core Debugging Components**

| Component         | Description                                |
| ----------------- | ------------------------------------------ |
| Execution Trace   | Complete ordered history of node execution |
| State Snapshots   | State at each step                         |
| Checkpoints       | Persisted execution states                 |
| Interrupt Nodes   | Pausing mechanism                          |
| Replay Engine     | Deterministic re-execution                 |
| Visualization     | Graph & flow rendering                     |
| Human-in-the-loop | Manual intervention                        |

---

### **3. Enabling Interactive Debugging**

```python
result = graph.invoke(
    input,
    config={
        "recursion_limit": 20,
        "configurable": {"debug": True}
    }
)
```

When debug mode is active, LangGraph records:

* Node entry & exit
* State diffs
* Conditional decisions
* Errors & retries

---

### **4. Inspecting Execution Step-by-Step**

```python
for step in graph.stream(input):
    print(step)
```

**Output structure:**

```python
{
  "node": "reason",
  "state": {...},
  "event": "node_complete"
}
```

This enables **live stepping** through the graph.

---

### **5. Interrupt & Manual State Injection**

### Defining an interruptible node

```python
def review_node(state):
    if state["risk"] > 0.8:
        raise Interrupt("Human review required")
    return state
```

### Resuming after correction

```python
graph.resume(thread_id, new_state={"risk": 0.2})
```

This allows **surgical correction** without restarting the workflow.

---

### **6. Execution Replay**

LangGraph stores **checkpoints** automatically.

```python
graph.replay(run_id)
```

Benefits:

* Deterministic debugging
* Failure reproduction
* Regression analysis

---

### **7. Visual Debugging**

```python
display(graph.get_graph().draw_mermaid())
```

Coupled with step traces, this gives a **complete execution narrative**.

---

### **8. Debugging in Cyclic & Agent Systems**

In loops and multi-agent flows, interactive debugging is critical:

| Failure Mode       | Debug Tool                 |
| ------------------ | -------------------------- |
| Infinite loops     | Step counter + breakpoints |
| Agent disagreement | Message inspection         |
| Bad routing        | Conditional edge trace     |
| Tool failures      | Retry + state rewind       |

---

### **9. Production Debugging Workflow**

```
Live Execution
   ↓
Trace + Metrics
   ↓
Checkpoint Snapshot
   ↓
Replay Failure
   ↓
Inject Fix
   ↓
Resume Execution
```

---

### **10. Benefits**

| Without Interactive Debugging | With Interactive Debugging |
| ----------------------------- | -------------------------- |
| Opaque failures               | Explainable failures       |
| Restart from scratch          | Resume from failure        |
| Blind execution               | Full observability         |
| Unsafe autonomy               | Controlled autonomy        |

---

### **Mental Model**

> LangGraph debugging treats execution as a **time-traveling state machine**
> where every decision, transition, and mutation is inspectable and reversible.


### Demonstration

In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

# -------------------------
# 1. Define State
# -------------------------
class State(TypedDict):
    step: int
    risk: float
    approved: bool

# -------------------------
# 2. Nodes
# -------------------------
def assess(state):
    print(f"Assessing... step={state['step']}, risk={state['risk']}")
    return {"step": state["step"] + 1}

def evaluate(state):
    if state["risk"] > 0.7:
        print("⚠️  High risk detected. Pausing for human review.")
        raise Exception("INTERRUPT")   # simulate interrupt
    return {"approved": True}

def reduce_risk(state):
    print("Reducing risk...")
    return {"risk": state["risk"] - 0.4}

# -------------------------
# 3. Router
# -------------------------
def router(state):
    if state.get("approved"):
        return END
    return "reduce"

# -------------------------
# 4. Build Graph
# -------------------------
builder = StateGraph(State)

builder.add_node("assess", assess)
builder.add_node("evaluate", evaluate)
builder.add_node("reduce", reduce_risk)

builder.set_entry_point("assess")
builder.add_edge("assess", "evaluate")
builder.add_conditional_edges("evaluate", router, {"reduce": "reduce", END: END})
builder.add_edge("reduce", "assess")   # cycle

graph = builder.compile()

# -------------------------
# 5. Run with Interactive Debug
# -------------------------
state = {"step": 0, "risk": 0.9, "approved": False}

try:
    for update in graph.stream(state):
        print("TRACE:", update)
except Exception:
    print("\n🛑 Execution paused. Injecting human fix...\n")
    state["risk"] = 0.3
    for update in graph.stream(state):
        print("TRACE:", update)


Assessing... step=0, risk=0.9
TRACE: {'assess': {'step': 1}}
⚠️  High risk detected. Pausing for human review.

🛑 Execution paused. Injecting human fix...

Assessing... step=0, risk=0.3
TRACE: {'assess': {'step': 1}}
TRACE: {'evaluate': {'approved': True}}
